<a href="https://colab.research.google.com/github/TottiPuc/Machine_learning/blob/master/An%C3%A1lisis_sentimiento_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Análisis de sentimientos en reseñas de peliculas usando Pytorch
En este notebook usaremos pytorch para realizar tareas de PLN y crear una red neuronal que pueda extraer las caracteristicas de un texto y predecir si es un comentario bueno o malo sobre una pelicula.
Usaremos el dataset publico de imdb disponible en [dataset_peliculas](https://www.imdb.com/interfaces/) 

In [0]:
!pip install watermark

In [0]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import sklearn
from torch.utils.data import DataLoader, Dataset
from sklearn.feature_extraction.text import CountVectorizer
from tqdm.notebook import tqdm, tqdm_notebook

In [4]:
%reload_ext watermark
%watermark -a "Christian Dayan Arcos Gordillo" --iversion

pandas  1.0.3
sklearn 0.22.2.post1
torch   1.4.0
Christian Dayan Arcos Gordillo


In [0]:
#definiendo el trabajo con GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')